In [ ]:
source = "CombineSignals";

In [ ]:
using NBInclude
import XGBoost
@nbinclude("Alpha.ipynb");

In [ ]:
@nbinclude("../TrainingAlphas/XGBoostFeatures.ipynb");

In [ ]:
# used for XGBoostFeatures
function read_params(alpha)
    params = load("../../data/alphas/$alpha/params.jld2")
    if alpha == "UserItemBiases"
        μ = mean(read_recommendee_alpha("UserItemBiases").rating - params["a"])
        push!(params["u"], μ)
    end
    params
end;

In [ ]:
function compute_alpha()
    params = read_params(source);
    X = get_augmented_indep("recommendee_inference", params["alphas"], params["β"]);
    bst = XGBoost.Booster(model_file = params["bst"])
    boost_preds = XGBoost.predict(bst, X)
    predictions =
        get_indep("recommendee_inference", params["alphas"]) * params["β"] + boost_preds;
    @debug "average bst magnitude: $(mean(abs.(boost_preds))), $(sqrt(mean(boost_preds .^ 2)))"
    
    # predictions
    model(items) = predictions[items]
    write_recommendee_alpha(model);
    
    # error predictions
    error_bst = XGBoost.Booster(model_file = params["error_bst"])
    error_preds = clamp.(XGBoost.predict(error_bst, hcat(X, boost_preds)), 0, Inf);
    model(items) = error_preds[items]
    write_recommendee_alpha(model, outdir = "PredictedErrors");    
end

In [ ]:
compute_alpha()